In [71]:
import tensorflow as tf
import sys
print(sys.version)
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

2.7.11 (default, Jan 22 2016, 08:29:18) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.2 (clang-700.1.81)]


In [83]:
class MF_RS():
    def __init__(self, numUsers, numSongs, embedding_dim):
        
        #hyper parameters
        self.batch_size = 32;
        self.numUsers = numUsers
        self.numSongs = numSongs
        self.epochs = 100
        
        #embedding matricies for users and songs
        self.userMat = tf.Variable(tf.random_normal([numUsers, embedding_dim]))
        self.songMat = tf.Variable(tf.random_normal([numSongs, embedding_dim]))
        
        #input tensors for songs, usres, ratings
        self.users = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.songs = tf.placeholder(tf.int32, shape =(self.batch_size))
        self.rating = tf.placeholder(tf.float32, shape = (self.batch_size))
        
        print(self.batch_size)
        print(self.users.get_shape())
        
        #map each user/song to its feature vector
        self.U = tf.nn.embedding_lookup(self.userMat, self.users)
        self.W = tf.nn.embedding_lookup(self.songMat, self.songs)
        
        #predicted rating is dot product of user and song
        self.yhat = tf.reduce_sum(tf.mul(self.U, self.W), 1)
        
        self.cost = tf.reduce_mean(tf.nn.l2_loss(self.yhat - self.rating))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate = .01).minimize(self.cost)
        
        self.session = tf.Session()
        self.session.run(tf.initialize_all_variables())
        
        
    def train(self, users, songs, ratings):
        
        for i in range(self.epochs):
            
            avg_cost = 0
            perm = np.random.permutation(len(ratings))
            num_batches = len(ratings) // self.batch_size
            
            for b_idx in range(num_batches):
                
                batch = perm[self.batch_size * b_idx:self.batch_size * (b_idx + 1)]
                #if b_idx == num_batches - 1:
                #    batch = perm[self.batch_size * b_idx:]
                    
                users_batch = users[batch]
                songs_batch = songs[batch]
                ratings_batch = ratings[batch]
                                
                avg_cost += self.session.run([self.cost, self.optimizer],
                                  {self.users:users_batch, self.songs:songs_batch, self.rating:ratings_batch})[0]
                
            print(avg_cost/num_batches)

In [74]:
a = np.array([1, 2, 3, 4, 5])
b = np.array([1, 2, 3, 4, 5])
c = np.array([4, 3, 2, 5, 1])
model = MF_RS(10, 10, 7)
for i in range(10):
    print(model.session.run([model.cost, model.optimizer], {model.users:a, model.songs:b, model.rating:c}))


32
(32,)


ValueError: Cannot feed value of shape (5,) for Tensor u'Placeholder_27:0', which has shape '(32,)'

In [ ]:
model.train(a, b, c)

In [75]:
movieratings = pd.read_csv('ratings.csv')

In [6]:
print(len(movieratings['userId'].unique()))
print(len(movieratings['movieId'].unique()))

671
9066


In [102]:
model = MF_RS(10, 10, 7)

In [9]:
movieratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [104]:
def getDfSummary(input_data):
    output_data = input_data.describe(include = 'all').T
    var = pd.DataFrame(data = {'nanvals': pd.Series(), 'number_distinct': pd.Series()})
    for i in range(len(input_data.columns)):
        nanvals = input_data.ix[:,i].isnull().sum()
        number_distinct = len(input_data.ix[:,i].value_counts())
        var = var.append(pd.DataFrame([[nanvals, number_distinct]], columns = ['nanvals', 'number_distinct']))
    var.index = output_data.index.values
    output_data['nanvals'] = var['nanvals']
    output_data['number_distinct'] = var['number_distinct']
    return output_data
output_data = getDfSummary(movieratings)

In [12]:
movieratings.keys()

Index([u'userId', u'movieId', u'rating', u'timestamp'], dtype='object')

In [76]:


users = movieratings.ix[:,0].values
songs = movieratings.ix[:,1].values
ratings = movieratings.ix[:,2].values

#unique users / songs
uni_users = movieratings['userId'].unique()
uni_songs = movieratings['movieId'].unique()

#dict mapping the id to an index
user_map = dict(zip(uni_users,range(len(uni_users))))
song_map = dict(zip(uni_songs,range(len(uni_songs))))

user_idx =  np.array([ user_map[user] for user in users])
song_idx =  np.array([ song_map[song] for song in songs])

print(len(uni_users),len(uni_songs))

(671, 9066)


In [84]:
songmodel = MF_RS(671, 9066, 40)

32
(32,)


In [85]:
print(len(users), len(songs), len(ratings))

(100004, 100004, 100004)


In [86]:
songs

array([  31, 1029, 1061, ..., 6365, 6385, 6565])

In [87]:
#this is failing
songmodel.train(user_idx, song_idx, ratings)

365.570101409
88.200579046
50.4981575562
36.3291015169
28.6065008434
23.8210150513
20.5856171742
18.1478148335
16.3645008325
14.9010023604
13.7862070777
12.8929791943
12.1411580664
11.5106896699
10.9725387325
10.5097424044
10.1399679417
9.78050922203
9.48333833755
9.20764265724
8.97081208511
8.74047586357
8.54792095985
8.37095146149
8.21362816162
8.07700348679
7.9295162323
7.80480015999
7.66907253929
7.56623234314
7.44958067314
7.35752015594
7.27520633469
7.18532850525
7.09148278984
7.01337833954
6.94039282646
6.8578497102
6.78813781105
6.73093115326
6.63652485008
6.56945139549
6.52454829315
6.47052772118
6.39915865395
6.35311679665
6.27134874302
6.24487279404
6.1737092247
6.11833489597
6.08030843307
6.02402429367
5.95926027504
5.91918171127
5.87384805351
5.8332385004
5.78467549381
5.73836539284
5.67780501648
5.63368910324
5.59226052895
5.55286458435
5.49063182781
5.4739006694
5.41620777607
5.36652687717
5.33148682701
5.28749066677
5.25612674183
5.21758364784
5.1738542754
5.12415341789

In [30]:
songmodel.songs.get_shape()

TensorShape([Dimension(32)])

In [61]:

# [ 2858  7190 57972  2105  2683] [ 4.5  4.   4.   3.5  3. ]
a = np.array([606 ,547, 468, 654 ,304])
b = np.array([ 2858,  7190, 57972,  2105 , 2683] )
c = np.array([ 4.5 , 4.,   4.,   3.5,  3. ])

songmodel.session.run([songmodel.cost, songmodel.optimizer],
                                  {songmodel.users:600*np.ones(32), songmodel.songs:np.ones(32), songmodel.rating:np.ones(32)})[0]

#model.session.run([model.yhat],
#                                  {model.users:a, model.songs:b, model.rating:c})[0]

2738.811

In [32]:
print(tf.all_variables())

[<tensorflow.python.ops.variables.Variable object at 0x10d621c10>, <tensorflow.python.ops.variables.Variable object at 0x111598790>, <tensorflow.python.ops.variables.Variable object at 0x111598d10>, <tensorflow.python.ops.variables.Variable object at 0x106c73e90>, <tensorflow.python.ops.variables.Variable object at 0x111652d90>, <tensorflow.python.ops.variables.Variable object at 0x106c9fad0>, <tensorflow.python.ops.variables.Variable object at 0x106c9f650>, <tensorflow.python.ops.variables.Variable object at 0x110fe1550>]


In [88]:
users

array([  1,   1,   1, ..., 671, 671, 671])

In [92]:
np.array([users,songs]).transpose().shape

(100004, 2)